In [2]:
# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
#Import required packages
# import dataiku
# from dataiku import pandasutils as pdu
import pandas as pd
import numpy as np
from sap_ibp_connector import KeyfigureDataDownloader, MasterdataDownloader, KeyfigureDataUploader, IBPConnector
from dateutil.relativedelta import relativedelta
import datetime
import os

# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
from sap_ibp_connector.query import (
    Query,
    filter,
    select,
    skip,
    top,
    order_by,
    AND,
    OR,
    OrderDirections,
    string_to_query_date,
    py_datetime_to_query_date,
)

C:\Users\A4023862\Anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [3]:
# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
#Example of reading key-figure data

# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
# DEV CREDENTIALS
# IBP_USERNAME_MD_KF = "ODATA_TECH_USER"
# IBP_PASSWORD_MD_KF = "DtEzcNUBacYBdCZZQaiwyQo5otJppqxkWCz>VhVP"
# IBP_URL_KF = "https://my400714-api.scmibp.ondemand.com/sap/opu/odata/IBP/PLANNING_DATA_API_SRV"
# IBP_PLANNING_AREA = "ZAPPLESI3"

# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
# User Credentials of Production system
IBP_USERNAME_MD_KF = "ODATA_TECH_USER"
IBP_PASSWORD_MD_KF = "mktxcwNnhRFsbgJfYzz/tAULCJP5UrwNhzKjRSYx"
IBP_URL_KF = "https://my400774-api.scmibp.ondemand.com/sap/opu/odata/IBP/PLANNING_DATA_API_SRV"
IBP_PLANNING_AREA = "ZAPPLESI3"

# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
# You can always call the help function which will provide information regarding the passed object
#help(KeyfigureDataDownloader)

# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
curr_date = datetime.datetime.now().date()
curr_month = datetime.datetime.now().replace(day=1).date()
actual_start_date = curr_month-relativedelta(months=60)
actual_end_date = curr_month-relativedelta(months=1)
print("Current Month: ",curr_month)
print("Actual Start Month: ",actual_start_date)
print("Actual End Month: ",actual_end_date)

Current Month:  2022-11-01
Actual Start Month:  2017-11-01
Actual End Month:  2022-10-01


In [ ]:
# -------------------------------------------------------------------------------- NOTEBOOK-CELL: CODE
# Mira 9/14/2022: Corrected, use query on line 15
# Set up connection using IBPConnector
with IBPConnector(
    username=IBP_USERNAME_MD_KF,
    password=IBP_PASSWORD_MD_KF,
    planning_area=IBP_PLANNING_AREA,
    url=IBP_URL_KF,
) as connector:

    # Instantiate data downloader
    downloader = KeyfigureDataDownloader(connector)

    # Create a query
    #select_query = select("CLEANSEDSALESHISTORY","PRDID", "PRDDESCR", "LOCREGION", "CUSTBUFFER2", "LOCID", "LOCDESCR", "PRDGROUP", "PRDGROUPID", "BRAND", "BRANDID", "DOSAGEFORM", "DOSAGEFORMID", "DOSAGE", "DOSAGEID", "PACKAGINGTYPE", "PACKAGINGTYPEID", "SUPPLYCHAINCAT", "LOCBUFFER3", "PERIODID3_TSTAMP", "CONSENSUSDEMANDQTYLM", "CONSENSUSDEMANDQTY", "ACTUALSQTY", "FINALSTATFCSTQTY", "CDPQ0MLAG", "CDPQ1MLAG", "CDPQ2MLAG", "CDPQ3MLAG", "CDPQ4MLAG", "CDPQ5MLAG", "FREEOFCHARGESALES", "SUMACTFOC", "STATISTICALFORECASTQTY3MLAG", "ZFINALSTATFCSTQTY")
    select_query = select("CLEANSEDSALESHISTORY","PRDID", "PRDDESCR", "LOCID", "PRDGROUP", "PRDGROUPID", "SUPPLYCHAINCAT", "ACTUALSQTY", "CUSTCOUNTRY","PERIODID3_TSTAMP")


    filter_query = AND(
        filter.eq("UOMTOID", "PC"),
        filter.ge("PERIODID3_TSTAMP", string_to_query_date(str(actual_start_date), "%Y-%m-%d")),
        filter.le("PERIODID3_TSTAMP", string_to_query_date(str(actual_end_date), "%Y-%m-%d")),
    )

    query = Query(select_query, filter_query)
    query = select_query + filter_query
    #print(query)
    # Download data
    df, response = downloader.download_in_chunks(query=query)

In [6]:
df

,__metadata,CUSTCOUNTRY,LOCID,PRDDESCR,PRDGROUP,PRDGROUPID,PRDID,SUPPLYCHAINCAT,PERIODID3_TSTAMP,ACTUALSQTY,CLEANSEDSALESHISTORY,PERIODID3_DATETIME
0,{'id': 'https://my400774-api.scmibp.ondemand.c...,JP,JP53,HARNAL D TAB 0.2 40X14 JP,TAMSULOSIN TAB,405,00648,CX,/Date(1509494400000)/,1499.000000,1499.000000,2017-11-01
1,{'id': 'https://my400774-api.scmibp.ondemand.c...,JP,JP54,HARNAL D TAB 0.2 40X14 JP,TAMSULOSIN TAB,405,00648,CX,/Date(1509494400000)/,1291.000000,1291.000000,2017-11-01
2,{'id': 'https://my400774-api.scmibp.ondemand.c...,JP,JP55,HARNAL D TAB 0.2 40X14 JP,TAMSULOSIN TAB,405,00648,CY,/Date(1509494400000)/,0.000000,0.000000,2017-11-01
3,{'id': 'https://my400774-api.scmibp.ondemand.c...,JP,JP53,VESICARE OD TAB 2.5 10X10 JP,SOLIFENAC SUCCINATE,407,00721,BZ,/Date(1509494400000)/,6473.000000,6473.000000,2017-11-01
4,{'id': 'https://my400774-api.scmibp.ondemand.c...,JP,JP54,VESICARE OD TAB 2.5 10X10 JP,SOLIFENAC SUCCINATE,407,00721,BY,/Date(1509494400000)/,6446.000000,6446.000000,2017-11-01
...,...,...,...,...,...,...,...,...,...,...,...,...
270059,{'id': 'https://my400774-api.scmibp.ondemand.c...,PA,IE51,PROGRAF XL 1MG CAPS 10X5 BL PK INTL,TACROLIMUS ER,611,M067711,DIS,/Date(1664582400000)/,0.000000,0.000000,2022-10-01
270060,{'id': 'https://my400774-api.scmibp.ondemand.c...,PA,US51,PROGRAF XL 1MG CAPS 10X5 BL PK INTL,TACROLIMUS ER,611,M067711,DIS,/Date(1664582400000)/,0.000000,0.000000,2022-10-01
270061,{'id': 'https://my400774-api.scmibp.ondemand.c...,PA,IE51,PROGRAF XL 5 MG CAPS 10X5 BL PK INTL,TACROLIMUS ER,611,M068711,DIS,/Date(1664582400000)/,0.000000,0.000000,2022-10-01
270062,{'id': 'https://my400774-api.scmibp.ondemand.c...,PA,US51,PROGRAF XL 5 MG CAPS 10X5 BL PK INTL,TACROLIMUS ER,611,M068711,DIS,/Date(1664582400000)/,0.000000,0.000000,2022-10-01
